In [123]:
import pandas as pd
from tsl.data.spatiotemporal_dataset import SpatioTemporalDataset
from tsl.ops.connectivity import SparseTensor
import torch
import numpy as np
from torch_geometric.utils import to_undirected
from pathlib import Path


In [124]:
df = pd.read_parquet("/home/jaydenfassett/amlproject/data/county_data")
graph = pd.read_csv("/home/jaydenfassett/amlproject/data/geographic/graph.csv")
outage = pd.read_csv("/home/jaydenfassett/amlproject/data/outage/outage.csv")
outage

,run_start_time,0,1,2,3,4,5,6,7,8,...,3047,3048,3049,3050,3051,3052,3053,3054,3055,3056
0,2022-01-01 00:00:00,0.0,0.0,0.0,2.0,0.0,0.0,63.5,13.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2022-01-01 06:00:00,0.0,0.0,0.0,4.0,0.0,0.0,55.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2022-01-01 12:00:00,4.0,4.0,0.0,22.0,4.0,0.0,112.0,5.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,2022-01-01 18:00:00,42.0,33.0,0.0,38.0,4.0,0.0,235.0,6.0,930.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,2022-01-02 00:00:00,0.0,4.0,37.0,19.0,0.0,0.0,358.5,0.0,2708.5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,2022-12-30 18:00:00,0.0,4.0,2.0,0.0,4.0,0.0,14.0,4.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1456,2022-12-31 00:00:00,4.0,5.0,443.5,22.0,4.0,0.0,705.5,0.0,9.5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1457,2022-12-31 06:00:00,7.0,8.0,13.0,4.0,212.0,2.0,637.0,20.0,3.5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1458,2022-12-31 12:00:00,6.0,9.0,16.5,0.0,4.0,0.0,90.5,9.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [125]:
df

geopotential_at_surface  land_sea_mask  \
county datetime            level                                           
1001   2022-01-01 00:00:00 1                  1405.142578       0.996516   
                           2                  1405.142578       0.996516   
                           3                  1405.142578       0.996516   
                           5                  1405.142578       0.996516   
                           7                  1405.142578       0.996516   
...                                                   ...            ...   
78030  2022-01-11 06:00:00 900                 186.934738       0.069189   
                           925                 186.934738       0.069189   
                           950                 186.934738       0.069189   
                           975                 186.934738       0.069189   
                           1000                186.934738       0.069189   

                                  2m_temperature  mean_sea_level_pressure  \
county datetime            level                                            
1001   2022-01-01 00:00:00 1          296.282959            101213.914062   
                           2          296.282959            101213.914062   
                           3          296.282959            101213.914062   
                           5          296.282959            101213.914062   
                           7          296.282959            101213.914062   
...                                          ...                      ...   
78030  2022-01-11 06:00:00 900        297.569702            101670.617188   
                           925        297.569702            101670.617188   
                           950        297.569702            101670.617188   
                           975        297.569702            101670.617188   
                           1000       297.569702            101670.617188   

                                  10m_v_component_of_wind  \
county datetime            level                            
1001   2022-01-01 00:00:00 1                     4.151270   
                           2                     4.151270   
                           3                     4.151270   
                           5                     4.151270   
                           7                     4.151270   
...                                                   ...   
78030  2022-01-11 06:00:00 900                  -1.644775   
                           925                  -1.644775   
                           950                  -1.644775   
                           975                  -1.644775   
                           1000                 -1.644775   

                                  10m_u_component_of_wind  \
county datetime            level                            
1001   2022-01-01 00:00:00 1                     0.345775   
                           2                     0.345775   
                           3                     0.345775   
                           5                     0.345775   
                           7                     0.345775   
...                                                   ...   
78030  2022-01-11 06:00:00 900                  -4.635486   
                           925                  -4.635486   
                           950                  -4.635486   
                           975                  -4.635486   
                           1000                 -4.635486   

                                  total_precipitation_6hr  \
county datetime            level                            
1001   2022-01-01 00:00:00 1                     0.001033   
                           2                     0.001033   
                           3                     0.001033   
                           5                     0.001033   
                           7                     0.001033   
...                         

In [126]:
target = pd.read_csv("../data/eaglei_data/eaglei_outages_2022.csv")
adj = pd.read_csv("../data/geographic/graph.csv")
unique = pd.unique(np.concat((target["fips_code"].values.ravel(), adj[["src","dest"]].values.ravel())))
fips2idx = pd.Series(index=unique, data=range(len(unique)))
fips2idx

1005        0
1009        1
1051        2
1055        3
1057        4
         ... 
30069    3181
31007    3182
48403    3183
35021    3184
32017    3185
Length: 3186, dtype: int64

In [127]:
fips2idx.reset_index().set_index(0)


,index
0,
0,1005
1,1009
2,1051
3,1055
4,1057
...,...
3181,30069
3182,31007
3183,48403


In [128]:
outage = outage.rename(columns={"run_start_time":"datetime"})
outage["datetime"] = outage['datetime'].astype('datetime64[ns]')

In [129]:
# df1_reset = df.reset_index()

In [130]:
outages_long = outage.melt(id_vars='datetime', var_name='county', value_name='outage_count')
outages_long

,datetime,county,outage_count
0,2022-01-01 00:00:00,0,0.0
1,2022-01-01 06:00:00,0,0.0
2,2022-01-01 12:00:00,0,4.0
3,2022-01-01 18:00:00,0,42.0
4,2022-01-02 00:00:00,0,0.0
...,...,...,...
4463215,2022-12-30 18:00:00,3056,0.0
4463216,2022-12-31 00:00:00,3056,0.0
4463217,2022-12-31 06:00:00,3056,0.0
4463218,2022-12-31 12:00:00,3056,0.0


In [131]:
idx2fips = fips2idx.reset_index().set_index(0)['index']

outages_long['county'] = outages_long['county'].astype("int64").map(idx2fips)
outages_long = outages_long.set_index(['county','datetime'])

In [132]:
outages_long

outage_count
county datetime                         
1005   2022-01-01 00:00:00           0.0
       2022-01-01 06:00:00           0.0
       2022-01-01 12:00:00           4.0
       2022-01-01 18:00:00          42.0
       2022-01-02 00:00:00           0.0
...                                  ...
53059  2022-12-30 18:00:00           0.0
       2022-12-31 00:00:00           0.0
       2022-12-31 06:00:00           0.0
       2022-12-31 12:00:00           0.0
       2022-12-31 18:00:00           0.0

[4463220 rows x 1 columns]

In [133]:
dfr = df.reset_index()
dfr = dfr.set_index(["county",'datetime'])

In [134]:
merged = pd.merge(dfr, outages_long, left_index=True,right_index=True)  # or 'inner' or 'outer'

In [135]:
merged = merged.reset_index()
df = merged.set_index(['county','datetime','level'])

In [136]:

df = df.reset_index()
# mean 0, var 1 all feature columns

for col in df.columns.drop(labels=["level", "county", "datetime", "outage_count"]):
    print("Normalizing",col)
    assert col != "index", "Re-run notebook if index is in cols"
    df[col] = (df[col] - df[col].mean()) / df[col].std()

grouped = df.groupby(['datetime', 'county']).first().reset_index()
uniques = grouped.columns.drop(labels=["datetime","county"])
times = grouped['datetime'].unique()
nodes = grouped['county'].unique()

fips_idx = {name: i for i, name in enumerate(nodes)} # FIPS CODE / local index map.

T, N, F = len(times), len(nodes), len(uniques) - 1
features = grouped[[i for i in uniques]].values

datetimes = features[:,0].reshape(T, N)
tensor_3d = features[:,1:].reshape(T, N, F).astype(np.float32)

# t, n, f = tensor_3d


Normalizing geopotential_at_surface
Normalizing land_sea_mask
Normalizing 2m_temperature
Normalizing mean_sea_level_pressure
Normalizing 10m_v_component_of_wind
Normalizing 10m_u_component_of_wind
Normalizing total_precipitation_6hr
Normalizing toa_incident_solar_radiation
Normalizing temperature
Normalizing geopotential
Normalizing u_component_of_wind
Normalizing v_component_of_wind
Normalizing vertical_velocity
Normalizing specific_humidity


/tmp/ipykernel_2338441/1128857449.py:9: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  grouped = df.groupby(['datetime', 'county']).first().reset_index()


In [137]:
for col in ['src','dest']:
    graph[col] = graph[col].apply(lambda x: fips_idx[x])

In [152]:
x = torch.from_numpy(tensor_3d)
# target = x[:,:,-1]


# x = x[:,:,:-1]
E = torch.from_numpy(graph[['src','dest']].values).T
# E = tuple(graph[['src','dest']].values.T)

E_w = torch.from_numpy(graph["total_voltage"].values)

E_w = (E_w - E_w.mean()) / E_w.std()


In [ ]:
dataset = SpatioTemporalDataset(
    tensor_3d,                    
    mask=None,                    
    connectivity=(E,E_w),    # Takes edges & weights    
    horizon=1,        
    window=3,                     
    stride=1,            
    # target=tensor_3d.shape[-1] - 1 
)

# print(dataset)
# dataset.target = target
# print(dataset)

In [158]:
x.shape

torch.Size([42, 3233, 15])

In [156]:
for i in dataset:
    print(i.x.shape)
    break

torch.Size([3, 3233])


In [ ]:
y = x[:,:,-1]


x = x[:,:,:-1]

In [150]:
dataset.target

tensor([[ 0.,  1.,  0.,  ...,  0.,  0.,  0.],
        [ 0.,  0.,  0.,  ...,  0.,  0.,  0.],
        [ 0.,  1.,  4.,  ...,  0.,  0.,  0.],
        ...,
        [ 0.,  4.,  0.,  ..., 18.,  0.,  0.],
        [ 0.,  0.,  0.,  ..., 41.,  0.,  0.],
        [ 0.,  0.,  0.,  ...,  0.,  0.,  0.]])

In [143]:
savepath = Path("/home/jaydenfassett/amlproject/data/tsldataset")
savepath.mkdir(exist_ok=True,parents=True)

dataset.save(savepath / "dataset.tsl")